<a href="https://colab.research.google.com/github/perlatomdpi/NLP/blob/main/Sentiment_Analysis_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment analysis model with BERT**


# **Import dependencies**

In [ ]:
import numpy as np
import math
import re # advanced text pre-processing
import pandas as pd
from bs4 import BeautifulSoup # encode text
import random

from google.colab import drive # get data from drive

In [ ]:
# use tf2
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub # to download the weights of bert
from tensorflow.keras import layers # to create cnn layers
import bert

# **Data pre-processing**

In [ ]:
drive.mount("/content/drive")

cols = ["sentiment", "id", "query", "user", "text"]
data = pd.read.csv(
    "content/drive/My Drive/.../BERT/data/train.csv",
    header=None,
    names=cols,
    engine="python", 
    encoding="latin1"
)

data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True) # inplace garantee that new lighter data are loaded

# **Cleaning**

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", '',tweet)   # replace a substring with another
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", '', tweet)
    tweet = re.sub(r"[^A-Za-z.!?]", '', tweet)
    tweet = re.sub(r" +", '', tweet)
    return tweet

data_clean = [clean_tweet(tweet) for tweet in data.text]

data_labels = data.sentiment.values # get the value of the sentiment
data_labels[data_labels == 4] = 1   # in the data 4 is for positve and we converted it to 1

# **Tokenization**

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer # call bert module

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False) # create bert layer - hub is were the pre-trainined model is stored
                                             # here we use the light version of bert
                                             # L-12: 12 encoders
                                             # trainable=false --> we don't use bert for fine-tuning weights

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy() # give access to the vocab file as numpy
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy() # lower case
tokenizer = FullTokenizer(vocab_file, do_lower_case) # create the tokenizer

def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent)) # id: each token is related to a number

# per each sentence run tokenizer and give valid input too our model
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

# **Create dataset and use it as input for the trainin model**

The BERT model receives a fixed length of sentence as input. Usually the maximum length of a sentence depends on the data we are working on. For sentences that are shorter than this maximum length, we will have to add paddings (empty tokens) to the sentences to make up the length.

We will create padded batches (so we pad sentences for each batch independently), this way we add the minimum of padding tokens possible. For that, we sort sentences by length, apply padded_batches and then shuffle.

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)] # iterate over the data

random.shuffle(data_with_len) # shuffle it: first half is negative sentiment, second half is positive sentiment
                              # so with shuffle we have negative and positive sentences mixed and not only positve or negative

data_with_len.sort(key=lambda x: x[2]) # sort according to the length
                                       # per each element x --> [sent, data_labels[i], len(sent)] access to the len(sent)
                                       # so len(sent) is the criteria for sorting

sorted_all = [(sent_lab[0], sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7] # get sentence with 7 words as minimum


# create dataset via generator --> generator give element one after the other 
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32)) # interger because the token are the token id


BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ())) # None: add empty token to make up the length

NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE) # give the number of batches
NB_BATCHES_TEST = NB_BATCHES // 10                   # get 10 test
all_batched.shuffle(NB_BATCHES)                      # shuffle in order to mix positive and negative lables
test_dataset = all_batched.take(NB_BATCHES_TEST)     # create test data
train_dataset = all_batched.skip(NB_BATCHES_TEST)    # create train

# **Model building**
Here we use embedding representation of vectors that we can use for our model. <br>
The most important part of CNN is the Convolutional part. Here we moltiply the inpt list vector by the Feature Detector to obtain the so-called Feature Map. The key part is the Feature Detector and this is the part that we have to train. <br>
So, our CNN learns to detect or useful or irrelavant features by tuning the coefficient of the Feature Detection matrix also called Convulational Filters. Tipically more than one convolutional fiters are used.



In [ ]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,                             # convolutional fiter --> feature detector --> 50 of size 2, 50 of size 3, 50 of size 4
                 FFN_units=512,                             # number of hidden units of our dense layer
                 nb_classes=2,                              # level of the independent variable: positive, negative sentiment
                 dropout_rate=0.1,                          # shut-down neurons to prevent overfitting                         
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,       # --> feature detector --> 50 of size 2
                                          emb_dim)          # convert embedding to vector
        self.bigram = layers.Conv1D(filters=nb_filters,     # focus on two consecutive words --> Conv1D: shift vertically the feature detector
                                    kernel_size=2,          # 2 because is bi.gram
                                    padding="valid",        # when we have steps more than 1 during the feature detection iteration --> we could get our of the max range
                                    activation="relu")      # Rectified Linear Unit get-rid of negative values that will be set to zero 

        self.trigram = layers.Conv1D(filters=nb_filters,    # --> feature detector --> 50 of size 3
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        
        self.fourgram = layers.Conv1D(filters=nb_filters,   # --> feature detector --> 50 of size 4
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        
        self.pool = layers.GlobalMaxPool1D()                # create the layers that take the feature max per each feature detector 2,3,4
        
        
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")  # feedforward neural network part --> we need 2 dense layers 
        self.dropout = layers.Dropout(rate=dropout_rate)                 # we use drop-out to shut down neurons --> prevent overfitting



        # the number of dense layer is equal to the number of the level of the dependent variable 
        # here we have a sentiment analysis with 2 levels --> positive/negative
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid") # sigmoid is used when we have only 2 levels od dependent variable 
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax") # if we have more than 2 levels dependent variable the softmax function is used instead
    



    # the call funtion is used to make predition or to prain the model
    # with the parameter "training" we specify that we are in the training phase
    # note that the drop-out is only used during training and not when we make prediction
    def call(self, inputs, training):   
        x = self.embedding(inputs)    # call the embedding

        # create the set of convolutional layers:
        #
        # for each of our 50 feature detectors of size 2 (bi-gram) --> we get the maxium activation (pooling)
        x_1 = self.bigram(x)    # batch_size, nb_filters, seq_len-1)
        x_1 = self.pool(x_1)    # (batch_size, nb_filters)

        # for each of our 50 feature detectors of size 3 (tri-gram) --> we get the maxium activation (pooling)
        x_2 = self.trigram(x)   # batch_size, nb_filters, seq_len-2)
        x_2 = self.pool(x_2)    # (batch_size, nb_filters)

        # for each of our 50 feature detectors of size 4 (four-gram) --> we get the maxium activation (pooling)
        x_3 = self.fourgram(x)  # batch_size, nb_filters, seq_len-3)
        x_3 = self.pool(x_3)    # (batch_size, nb_filters)
        

        # concatenate all the results and apply them to the dense layer
        merged = tf.concat([x_1, x_2, x_3], axis=-1)    # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)                   # apply dense layer
        merged = self.dropout(merged, training)         # apply drop-out
        output = self.last_dense(merged)
        
        return output

        # Here we have built and define our CNN model
        # Now we are ready to train it and create an instance of this classs

# **Training**


In [ ]:
# select the parameter for the neural network
VOCAB_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
# create the neural network
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [ ]:
# we need to give to the neural network:
# 1 - an optimizer
# 2 - loss function
# 3 - accuracy function

if NB_CLASSES == 2:                                           # the following paramters are good for a 2 level classification
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",                            # use adaptive momentum --> it combines the Momentum and RMS prop in a single approach making AdaM a very powerful and fast optimizer
                 metrics=["accuracy"])
    
    
else:                                                          # the following paramters are good if we have more than 2 level classification
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [ ]:
# before calling the fit function and train our model we need to create a check point
# check point --> when we create a mdel we want to keep it --> and use it later !!!

checkpoint_path = "./drive/My Drive/projects/BERT/ckpt_bert_tok/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)   # max_to_keep: max number of check point we want to keep insie the folder
                                                                                  # in this case --> keep only the last recent model


if ckpt_manager.latest_checkpoint:                  # return the latest check point in our folder --> return none if there is not a check point yet
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [ ]:
# create a custom --> callback
# witht the --> callback --> the fit function execute the lines of code between each epochs

class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):                              # for every moment of the training:
        ckpt_manager.save()                                                # 1 - store the model at the end of each epochs
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [ ]:
# call the "fit" method and train the model

Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])   # run the model per each epoch and see the accuracy

Accuracy on test set = 88.5

# **Evaluate**

In [ ]:
results = Dcnn.evaluate(test_dataset)
print(results)

Accuracy on test set = 84.6

In [ ]:
# get a sentence in input and peform CNN to calculate the sentiment

def get_prediction(sentence):
    tokens = encode_sentence(sentence)                 # create the tokens of the sentence
    inputs = tf.expand_dims(tokens, 0)                 # model get as input a batch --> here we simulate the batch that contain only our sentence

    output = Dcnn(inputs, training=False)              # we don't apply training/drop-out to the sentence 

    sentiment = math.floor(output*2)                   # we have 2 levels (positive/negative) but 1 output
                                                       # the sigmoid goes from 0 to 1


    if sentiment == 0:
        print("Ouput of the model: {}\nPredicted sentiment: negative.".format(
            output))
    elif sentiment == 1:
        print("Ouput of the model: {}\nPredicted sentiment: positive.".format(
            output))

In [ ]:
get_prediction("This movie was pretty interesting.")